In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(RPostgres)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(reshape2)
library(ggrepel)
library(ggpubr)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
currencies <- Currencies()

In [ ]:
startDate <- as.Date('2019-01-01')

closestExpiries <- currencies$NseFuturesTimeSeries() %>%
    filter(TIME_STAMP >= startDate)
    group_by(TIME_STAMP) %>%
    summarize(EXPIRY = min(EXPIRY_DT)) %>%
    arrange(TIME_STAMP) %>%
    print()